In [18]:
# 지금까지 배운 내용을 기초로 tensorflow의 mnist example을
# CNN으로 구현해 보아요

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Data loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

# 전처리 단계 (결측치, 이상치, 정규화, feature engineering)
# mnist예제에서는 이미 전처리가 끝난 상태라서 따로 할게 없다!

# Model 정의
tf.reset_default_graph()

# 1. placeholder
X = tf.placeholder(shape=[None, 784], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)

# 버전에 따라 1.5 => keep_prob = 0.8  살리는 비율
#                  rate = 0.2 죽이는 비율

# 2. Convolution Layer
x_img = tf.reshape(X, [-1,28,28,1])
L1 = tf.layers.conv2d(inputs=x_img, filters=32,
                         kernel_size=[3,3], padding="SAME",
                         strides=1, activation=tf.nn.relu)

L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],
                                padding="SAME",
                                strides=2)

L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

# W1 = tf.Variable(tf.random_normal([3,3,1,32])) # filter
# L1 = tf.nn.conv2d(x_img, W1, strides=[1,1,1,1], # CNN(합성곱)
#                              padding="SAME")
# L1 = tf.nn.relu(L1)                              # relu
# L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],
#                                            padding="SAME")

L2 = tf.layers.conv2d(inputs=L1, filters=64,
                         kernel_size=[3,3],  padding="SAME",
                         strides=1, activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],
                                padding="SAME",
                                strides=2)

L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

## W2 = tf.Variable(tf.random_normal([3,3,32,64]))
## L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1],
##                              padding="SAME")
## L2 = tf.nn.relu(L2)
## L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1],
##                                            padding="SAME")


# 3. FC Layer (= Dense Layer) 
# Dense Layer 로 바꾸자

L2 = tf.reshape(L2,[-1, 7*7*64])

Dense1 = tf.layers.dense(inputs=L2, units=256,
                            activation=tf.nn.relu) # 밑에 3줄을 한번에 처리
Dense1 = tf.layers.dropout(inputs=Dense1, rate=dropout_rate)

# W3 = tf.get_variable("weight3", shape=[7*7*64, 256],
#                         initializer=tf.contrib.layers.xavier_initializer())
# b3 = tf.Variable(tf.random_normal([256]), name="bias3")
# _layer1 = tf.nn.relu(tf.matmul(L2, W3)+b3)
# layer1 = tf.nn.dropout(_layer1, keep_prob=dropout_rate)

Dense2 = tf.layers.dense(inputs=Dense1, units=128,
                            activation=tf.nn.relu) # 밑에 3줄을 한번에 처리
Dense2 = tf.layers.dropout(inputs=Dense2, rate=dropout_rate)

# W4 = tf.get_variable("weight4", shape=[256,512],
#                         initializer=tf.contrib.layers.xavier_initializer())
# b4 = tf.Variable(tf.random_normal([512]), name="bias4")
# _layer2 = tf.nn.relu(tf.matmul(layer1, W4)+b4)
# layer2 = tf.nn.dropout(_layer2, keep_prob=dropout_rate)

Dense3 = tf.layers.dense(inputs=Dense2, units=512,
                            activation=tf.nn.relu) # 밑에 3줄을 한번에 처리
Dense3 = tf.layers.dropout(inputs=Dense3, rate=dropout_rate)

H = tf.layers.dense(inputs=Dense3, units=10) # H를 한번에 만들어 버린다

# W5 = tf.get_variable("weight5", shape=[512,10],
#                         initializer=tf.contrib.layers.xavier_initializer())
# b5 = tf.Variable(tf.random_normal([10]), name="bias5")

# # H
# logit = tf.matmul(layer2, W5)+b5
# H = tf.nn.softmax(logit)

# cost function
cost = tf.losses.softmax_cross_entropy(Y, H) # 순서가 바뀐다
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit,
#                                                                    labels = Y))

#
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

#
sess = tf.Session()
sess.run(tf.global_variables_initializer())



Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [19]:
H

<tf.Tensor 'dense_4/BiasAdd:0' shape=(?, 10) dtype=float32>

In [17]:
%%time
# 학습
train_epoch = 30
batch_size = 100

for step in range(train_epoch):
    num_of_iter = int(mnist.train.num_examples/ batch_size)
    cost_val = 0
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([train, cost], 
                               feed_dict={ X : batch_x,
                                            Y : batch_y})
    if step % 3 == 0:
        print("Cost값 : {}".format(cost_val))

        
# 학습이 종료 되었으니 정확도를 측정

predict = tf.argmax(H, 1)
correct = tf.equal(predict, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))
print("정확도는 : {}".format(sess.run(accuracy, feed_dict={X : mnist.test.images,
                                                          Y : mnist.test.labels})))

Cost값 : 0.22042438387870789
Cost값 : 0.2108023762702942
Cost값 : 0.21726827323436737
Cost값 : 0.02184995450079441
Cost값 : 0.09577726572751999
Cost값 : 0.1975776106119156
Cost값 : 0.056538380682468414
Cost값 : 0.18843795359134674
Cost값 : 0.046463608741760254
Cost값 : 0.08279932290315628
정확도는 : 0.9664999842643738
Wall time: 21min 15s


In [ ]:
#############결국 우리의 MNIST 예제는 multinomial 예제다!
############# 이미지 1개에 대한 예측값이 
############# H의 도출값은 예) [0.5,0.3,0.2,0.001,0.99,0.44,....]

### ensenble은 모델을 여러개 만든다.(10개의 모델)
### 이미지 1개에 대한 각 모델의 예측값
### H1 -> [0.5,0.3,0.2,0.001,0.99,0.44,....]
### H2 -> [0.4,0.2,0.3,0.011,0.90,0.64]
### H3 -> [0.8,0.3,0.4,0.005,0.78,,,,,]
### .
### .
### .
### .
### Hn

### Sum=> [1.7,0.8,0.9,0.017,......]
### 한사람의 전문가한테 의견을 구하는게 아니라
### 여러사람의 전문가에게 의견을 구해서 가장 그럴듯한 대답으로 찾아내는 방식

### 프로그래밍 영역


In [23]:
# Kaggle MNIST deeplearning

# 필요한 module import

import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Data Loading

k_train_mnist = pd.read_csv("./data/digit-recognizer/train.csv")
k_test_mnist = pd.read_csv("./data/digit-recognizer/test.csv")

tf.reset_default_graph()

scaler = MinMaxScaler()
scaler.fit(k_train_mnist)

x_data = scaler.fit_transform(k_train_mnist.iloc[:,1:])
y_data = pd.get_dummies(k_train_mnist["label"]).values

pre_x_data = scaler.fit_transform(k_test_mnist.values)


# train / test 구분
train_num = int(x_data.shape[0] * 0.8)

# train data set
train_x_data = x_data[:train_num]
train_y_data = y_data[:train_num]

# test data set
test_x_data = x_data[train_num:]
test_y_data = y_data[train_num:]



In [48]:
# 함수로 모델 1개 만드는 로직 짜기


result = []
def make_H(X,Y,d_rate):
    tf.reset_default_graph()
    X = tf.placeholder(shape=[None, 784], dtype=tf.float32)
    Y = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    d_rate = tf.placeholder(dtype=tf.float32)
    
    x_img = tf.reshape(X, [-1,28,28,1])
    L1 = tf.layers.conv2d(inputs=x_img, filters=32,
                             kernel_size=[3,3], strides=1,
                             padding="SAME", activation=tf.nn.relu)
    L1 = tf.layers.max_pooling2d(inputs=L1, pool_size = [2,2],
                                   padding="SAME", strides=2)
    L1 = tf.layers.dropout(inputs=L1, rate=d_rate)
    
    L2 = tf.layers.conv2d(inputs=L1, filters=64,
                             kernel_size=[3,3], strides=1,
                             padding="SAME", activation=tf.nn.relu)
    L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],
                                    padding="SAME", strides=2)
    L2 = tf.layers.dropout(inputs=L2, rate=d_rate)
    
    Dl3 = tf.reshape(L2, [-1, 7*7*64])
    
    Dl3 = tf.layers.dense(inputs=Dl3, units=256, activation=tf.nn.relu)
    
    Dl4 = tf.layers.dense(inputs=Dl3, units=128, activation=tf.nn.relu)
    
    Dl5 = tf.layers.dense(inputs=Dl4, units=512, activation=tf.nn.relu)
    
    H = tf.layers.dense(inputs=Dl5, units=10)
    
    cost = tf.losses.softmax_cross_entropy(Y, H) # 순서가 바뀐다

    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    train_epoch = 30
    batch_size = 100

    for step in range(train_epoch):
        num_of_iter = int(train_x_data.shape[0]/batch_size)
        cost_val = 0
        for i in range(num_of_iter):
            start = i * batch_size
            end = start + batch_size
            cut_train_x = train_x_data[start:end]
            cut_train_y = train_y_data[start:end]
            _, cost_val = sess.run([train, cost], 
                                   feed_dict={ X : cut_train_x,
                                                Y : cut_train_y,
                                                 d_rate : 0.8})

        if step % 3 == 0:
            print("Cost값 : {}".format(cost_val))

    
        tmp = sess.run(H, feed_dict={X : test_x_data})
        result = result + tmp
            
    return result





In [49]:
for _ in range(1):
    a=make_H(train_x_data, train_y_data, 0.2)
    print(a)

Cost값 : 0.25770843029022217
Cost값 : 0.020579295232892036
Cost값 : 0.06677761673927307
Cost값 : 0.01443882379680872
Cost값 : 0.12112454324960709
Cost값 : 0.17724022269248962
Cost값 : 0.04085906967520714
Cost값 : 0.10043641179800034
Cost값 : 0.016445619985461235
Cost값 : 0.042957957834005356
[[7.6954727, -5.101029, -4.3286467, -2.8076994, -1.2830647, 2.3543415, 5.3073344, -3.5426598, 2.308227, -2.9392533], [16.94471, -14.081453, -2.9537275, -3.6962588, -2.9622986, -1.5341485, -1.0021567, -8.313989, 1.1892447, -4.5225606], [29.236557, -32.527355, -10.678991, -12.057344, -10.8120165, -15.958943, -23.468891, -16.069109, -8.816397, -17.166168], [20.418243, -22.493408, 0.76354086, 0.84865487, -11.092806, -15.209392, -1.5207148, -15.477078, -4.1384335, -15.488857], [22.097359, -26.415403, -3.7350209, -7.209144, -20.795237, -16.879614, -4.900503, -12.226725, -4.1675777, -17.12745], [19.178059, -22.028955, -10.316296, -10.046954, -9.583377, -16.447462, -1.2524173, -6.1927648, -1.0661488, -14.020364], [1

In [131]:
a[0]
b=[0,0,0,0,0,0,0,0,0,0]
for j in range(len(a)):
    for i in range(len(a[0])):
        b[i] += a[j][i]
print(b)
b = np.array(b)
b
# sess.run(b, feed_dict={X : test_x_data})


# b=tf.layers.dense(inputs=b, units=10)
# sess.run(b, feed_dict={ X : test_x_data})
# print(H, b)
                   
b=np.reshape(b,[1,10])
b
#Y.shape
new_predict = np.argmax(b, 1)
new_predict
new_y = np.argmax(test_y_data , 1)
correct = np.equal(new_predict, new_y)
accuracy = np.mean(np.cast(correct))
print("정확도는 : {}".format(sess.run(accuracy, feed_dict={X : test_x_data,
                                                          Y : test_y_data,
                                                              dropout_rate : 1})))
# np.argmax(a, axis=None, out=None)
# tf.argmax(
#     input,
#     axis=None,
#     name=None,
#     dimension=None,
#     output_type=tf.int64,)

[1036.697193145752, -1012.5306437015533, -157.69621840119362, -1229.557319521904, -523.3045591711998, -911.7403128147125, 23.20236837863922, -956.1128082275391, 17.87038677930832, -275.16849333047867]


TypeError: '_typedict' object is not callable

In [ ]:
%%time
# 학습시켜보기! 
# batch function
train_epoch = 10
batch_size = 100

x = 

for i in range(10):
    make_H()
    
for step in range(train_epoch):
    num_of_iter = int(train_x_data.shape[0]/batch_size)
    cost_val = 0
    for i in range(num_of_iter):
        start = i * batch_size
        end = start + batch_size
        cut_train_x = train_x_data[start:end]
        cut_train_y = train_y_data[start:end]
        _, cost_val = sess.run([train, cost], 
                               feed_dict={ X : cut_train_x,
                                            Y : cut_train_y,
                                             drop_rate : 0.8})
    
        
    if step % 3 == 0:
        print("Cost값 : {}".format(cost_val))
      